In [ ]:
### install dependencies:
 
# might need to install opencv
!pip3 install Cython cupy-cuda112 cupy-cuda102   # Probably only need one of these but it works so I'm gonna leave it for now.
!pip3 install pyyaml==5.1
 
# Torch 1.8.1 does not work despite being the latest stable release. As such, use 1.7. 
# Cuda version on MAGEOHub is currently 11.3 - cu110 all works fine.
 
# To be tried once the bug with torch 1.8 has been fixed 
#!pip3 install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
 
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!python -m pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html

     |████████████████████████████████| 75.9 MB 56 kB/s 
     |████████████████████████████████| 60.6 MB 27 kB/s 
     |████████████████████████████████| 274 kB 4.2 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44091 sha256=33354abb1ef4c62c0869f2a7f12107fdcabc950e8305b8ce5e8cbdcd65e4538f
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 53.4 MB/s eta 0:00:07tcmalloc: large alloc 1147494400 bytes == 0x5561a34d4000 @  0x7f9550550615 0x55616a94002c 0x55616aa2017a 0x55616a942e4d 0x55616aa34c0d 0x55616a9b70d8 0x55616a9b1c35 0x55616a94473a 0x55616a9b6f40 0x55616a9b1c35 0x55616a94473a 0x55616a9b393b 0x55616aa

In [ ]:
# Import torch and check versions and cuda availability, and resources
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
!nvcc --version
!nvidia-smi

from IPython.display import display, clear_output

1.7.1+cu110 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
Sat Aug  7 12:57:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|======================

In [ ]:
# Install wandb
%%capture
!pip install wandb --upgrade

In [ ]:
# Or with this?
#!pip install wandb -qqq
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# In order to log data to our web service, you'll need to log in
import wandb
wandb.login()

wandb: Currently logged in as: detectree (use `wandb login --relogin` to force relogin)


True

In [ ]:
# Some basic imports:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# tensorboard?
#%load_ext tensorboard
#%tensorboard --logdir output

# necessary libraries
import pandas as pd
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
import json

#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [ ]:
# this mounts your google drive, so you can access files in it

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Make sure the inevitable error messages are useful.
CUDA_LAUNCH_BLOCKING="1"

In [ ]:
#### This loop gets the labels and images into the correct form for training.

def get_tree_dicts(directory):
    classes = ['tree']
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.geojson')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)
        
        
        
        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        # Make sure we have the correct height and width
        height, width = cv2.imread(filename).shape[:2]

        record["file_name"] = filename
        record["height"] = height
        record["width"] = width
        record["image_id"] = filename[0:400]
        print(filename[0:400])
            
        

        objs = []
        for features in img_anns['features']:
            anno = features['geometry']
            # print("##### HERE IS AN ANNO #####", anno)...weirdly sometimes (but not always) have to make 1000 into a np.array
            px = [a[0] for a in anno['coordinates'][0]]
            py = [np.array(height) - a[1] for a in anno['coordinates'][0]]
            # print("### HERE IS PY ###", py)
            poly = [(x, y) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]
            # print("#### HERE ARE SOME POLYS #####", poly)
            obj = {
                   "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                   "bbox_mode": BoxMode.XYXY_ABS,
                   "segmentation": [poly],
                   "category_id": 0,
                   "iscrowd": 0
                   }
            objs.append(obj)
            # print("#### HERE IS OBJS #####", objs)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

############################
#CHANGE THE FILENAMES HERE!#   
train_filename = "train_All"
test_filename = "test_Combined"
############################

# Remember to change the file directory as well!
from detectron2.data import DatasetCatalog, MetadataCatalog
for d in [train_filename, test_filename]:
    DatasetCatalog.register("trees_" + d, lambda d=d: get_tree_dicts('/content/drive/MyDrive/train_and _test_data/Week_4_5_dataset/' + d))
    MetadataCatalog.get("trees_" + d).set(thing_classes=['tree'])
trees_metadata = MetadataCatalog.get("trees_" + d)

In [ ]:
# Check if the file exists
os.path.isdir("/content/drive/MyDrive/train_and _test_data/Week_4_5_dataset/" + train_filename)

True

In [ ]:
### Setup the evaluator

from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    verify_results,
)

from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader


In [ ]:
# This block allows us to print loss on the test set as training goes on.

from detectron2.engine.hooks import HookBase
from detectron2.evaluation import inference_context
from detectron2.utils.logger import log_every_n_seconds
from detectron2.data import DatasetMapper, build_detection_test_loader, build_detection_train_loader
import detectron2.utils.comm as comm
#import torch
import time
import datetime
import logging
class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._period = eval_period
        self._data_loader = data_loader
    
    def _do_loss_eval(self):
        # Copying inference_on_dataset from evaluator.py
        total = len(self._data_loader)
        num_warmup = min(5, total - 1)
            
        start_time = time.perf_counter()
        total_compute_time = 0
        losses = []
        for idx, inputs in enumerate(self._data_loader):            
            if idx == num_warmup:
                start_time = time.perf_counter()
                total_compute_time = 0
            start_compute_time = time.perf_counter()
            if torch.cuda.is_available():
                torch.cuda.synchronize()
            total_compute_time += time.perf_counter() - start_compute_time
            iters_after_start = idx + 1 - num_warmup * int(idx >= num_warmup)
            seconds_per_img = total_compute_time / iters_after_start
            if idx >= num_warmup * 2 or seconds_per_img > 5:
                total_seconds_per_img = (time.perf_counter() - start_time) / iters_after_start
                eta = datetime.timedelta(seconds=int(total_seconds_per_img * (total - idx - 1)))
                log_every_n_seconds(
                    logging.INFO,
                    "Loss on Validation  done {}/{}. {:.4f} s / img. ETA={}".format(
                        idx + 1, total, seconds_per_img, str(eta)
                    ),
                    n=5,
                )
            loss_batch = self._get_loss(inputs)
            losses.append(loss_batch)
            wandb.log({"batch_loss":loss_batch})
        mean_loss = np.mean(losses)
        wandb.log({"mean_loss":mean_loss})
        self.trainer.storage.put_scalar('validation_loss', mean_loss)
        comm.synchronize()

        return losses
            
    def _get_loss(self, data):
        # How loss is calculated on train_loop 
        metrics_dict = self._model(data)
        metrics_dict = {
            k: v.detach().cpu().item() if isinstance(v, torch.Tensor) else float(v)
            for k, v in metrics_dict.items()
        }
        total_losses_reduced = sum(loss for loss in metrics_dict.values())
        return total_losses_reduced
        
        
    def after_step(self):
        next_iter = self.trainer.iter + 1
        is_final = next_iter == self.trainer.max_iter
        if is_final or (self._period > 0 and next_iter % self._period == 0):
            self._do_loss_eval()
        self.trainer.storage.put_scalars(timetest=12)

In [ ]:
# This is repeated below - this shouldn't matter though. This is just instantiating the config. 
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("trees_" + train_filename,)
cfg.DATASETS.TEST = ("trees_"+ test_filename,)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 900
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

In [ ]:
import detectron2.data.transforms as T
from detectron2.data import DatasetMapper   # the default mapper

dataloader = build_detection_train_loader(cfg,
        mapper=DatasetMapper(cfg, is_train=True, augmentations=[
        T.Resize((800, 800)), 
        T.RandomBrightness(0.8, 1.8),
        T.RandomContrast(0.6, 1.3),
        T.RandomSaturation(0.8, 1.4),
        T.RandomRotation(angle=[90, 90], expand=False),
        T.RandomLighting(0.7),
        T.RandomFlip(prob=0.4, horizontal=False, vertical=True)]))
# use this dataloader instead of the default

[08/07 13:21:09 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [Resize(shape=(800, 800)), RandomBrightness(intensity_min=0.8, intensity_max=1.8), RandomContrast(intensity_min=0.6, intensity_max=1.3), RandomSaturation(intensity_min=0.8, intensity_max=1.4), RandomRotation(angle=[90, 90], expand=False), RandomLighting(scale=0.7), RandomFlip(prob=0.4, horizontal=False, vertical=True)]
/content/drive/MyDrive/train_and _test_data/Week_4_5_dataset/train_All/202105_5cm_tile_286755_583730.png
/content/drive/MyDrive/train_and _test_data/Week_4_5_dataset/train_All/202105_5cm_tile_286555_583760.png
/content/drive/MyDrive/train_and _test_data/Week_4_5_dataset/train_All/2019_10cm_tile_287150_583760.png
/content/drive/MyDrive/train_and _test_data/Week_4_5_dataset/train_All/2019_10cm_tile_286755_583730.png
/content/drive/MyDrive/train_and _test_data/Week_4_5_dataset/train_All/202104_5cm_tile_286920_583980.png
/content/drive/MyDrive/train_and _test_data/Week_4_5_dataset/train_

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
import detectron2.data.transforms as T

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("eval_2", exist_ok=True)
            output_folder = "eval_2"
        return COCOEvaluator(dataset_name, cfg, True, output_folder)
                     
    def build_hooks(self):
        hooks = super().build_hooks()
        hooks.insert(-1,LossEvalHook(
            cfg.TEST.EVAL_PERIOD,
            self.model,
            build_detection_test_loader(
                self.cfg,
                self.cfg.DATASETS.TEST[0],
                DatasetMapper(self.cfg,True)
            )
        ))
        return hooks

    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=DatasetMapper(cfg, is_train=True, augmentations=[T.Resize((800, 800)), T.RandomBrightness(0.8, 1.8), 
        T.RandomContrast(0.6, 1.3), T.RandomSaturation(0.8, 1.4), T.RandomRotation(angle=[90, 90], expand=False), T.RandomLighting(0.7),
        T.RandomFlip(prob=0.4, horizontal=False, vertical=True)]))

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

# Define hyperparameters
# All hyperparameters should be defined here then called from dictionary where required
hyperparameters = dict(
    dl_num_workers=2,
    ims_per_batch=2,
    gamma=0.1,
    backbone_freeze_at=3,
    warmup_iters =120,
    momentum =0.9,
    batch_size_per_image =1024,
    weight_decay =0.001,
    base_lr=0.00025,
    max_iter=2000
)


def train(config=None):
  with wandb.init(project="tune", entity="detectree", config=hyperparameters,
           sync_tensorboard=True,
           settings=wandb.Settings(start_method="thread", console="off")):
    config=wandb.config

    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = ("trees_" + train_filename,)
    cfg.DATASETS.TEST = ("trees_"+ test_filename,)
    cfg.DATALOADER.NUM_WORKERS = config["dl_num_workers"]
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
    cfg.SOLVER.IMS_PER_BATCH = config["ims_per_batch"]
    cfg.SOLVER.BASE_LR = config["base_lr"]
    cfg.SOLVER.MAX_ITER = config["max_iter"]
    cfg.SOLVER.GAMMA = config["gamma"]
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = config["batch_size_per_image"]
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.MODEL.BACKBONE.FREEZE_AT = config["backbone_freeze_at"]
    cfg.INPUT.RANDOM_FLIP = "horizontal"
    cfg.SOLVER.WARMUP_ITERS = config["warmup_iters"]
    cfg.SOLVER.MOMENTUM = config["momentum"]
    cfg.SOLVER.WEIGHT_DECAY = config["weight_decay"]

    ### From here is the important bit that hasn't been repeated further up
    cfg.TEST.EVAL_PERIOD = 50
    
    
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    trainer = MyTrainer(cfg) 
    trainer.resume_or_load(resume=False)
    trainer.train()

# Change the numeber of iterations for training here
wandb.agent("0ohk40wx", train, project="tune", entity="detectree", count=100)

wandb: Agent Starting Run: y4xqv6gz with config:
wandb: 	base_lr: 6.82935181278687e-05
wandb: 	max_iter: 1466
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


[08/07 13:24:44 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_a3ec72.pkl: 254MB [00:25, 9.79MB/s]                           
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to i

[08/07 13:26:55 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.7/dist-packages/detectron2/structures/masks.py:363: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  item = item.nonzero().squeeze(1).cpu().numpy().tolist()
/usr/local/lib/python3.7/dist-packages/detectron2/structures/masks.py:363: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  item = item.nonzero().squeeze(1).cpu().numpy().tolist()
/usr/local/lib/python3.7/dist-packages/detectron2/modeling/roi_heads/fast_rcnn.py:103: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/

...

global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██████████████
rpn/num_pos_anchors,▅▅▇▅▃▅▅▅▃▄▃▃▆▅▅▆▇▄▆▄▅▆█▅▇▅▃▅▅▇▃▃▅▅▇▅▆▇▄▁
rpn/num_neg_anchors,▄▄▂▄▆▄▄▃▆▅▆▆▃▄▄▃▂▅▃▅▄▃▁▄▂▄▆▄▄▁▆▆▄▄▂▄▃▂▅█
roi_head/num_fg_samples,▁▃▆▆▆▇▆▇▇▇▇████████████████████████████▇
roi_head/num_bg_samples,█▇▆▅▅▄▅▄▅▄▅▄▃▄▄▃▃▃▃▃▃▃▂▃▃▂▃▃▃▂▂▃▂▂▁▁▁▂▂▆
fast_rcnn/cls_accuracy,▃▂▁▃▃▃▃▄▄▃▄▅▅▄▅▅▅▅▅▅▆▆▆▆▅▆▇▇▇▆▇█▇▇██▇▇█▂
fast_rcnn/fg_cls_accuracy,▁▁▁▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇██▇█████████▆
fast_rcnn/false_negative,███▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▃
mask_rcnn/accuracy,▁▂▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇██▇██████▆
mask_rcnn/false_positive,▅█▃▂▂▂▂▂▂▂▂▁▂▁▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂


wandb: Agent Starting Run: cywd9uct with config:
wandb: 	base_lr: 0.0003303972148877813
wandb: 	max_iter: 1980
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


[08/07 14:29:16 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

[08/07 14:29:24 d2.engine.train_loop]: Starting training from iteration 0
[08/07 14:29:36 d2.utils.events]:  eta: 0:18:17  iter: 19  total_loss: 2.562  loss_cls: 0.5981  loss_box_reg: 0.4397  loss_mask: 0.692  loss_rpn_cls: 0.5868  loss_rpn_loc: 0.1823  time: 0.5622  data_time: 0.0369  lr: 5.2591e-05  max_mem: 4576M
[08/07 14:29:48 d2.utils.events]:  eta: 0:18:55  iter: 39  total_loss: 2.251  loss_cls: 0.5205  loss_box_reg: 0.5968  loss_mask: 0.6497  loss_rpn_cls: 0.2704  loss_rpn_loc: 0.2  time: 0.5853  data_time: 0.0065  lr: 0.0001076  max_mem: 4576M
/content/drive/MyDrive/train_and _test_data/Week_4_5_dataset/test_Combined/202104_5cm_tile_287185_584065.png
/content/drive/MyDrive/train_and _test_data/Week_4_5_dataset/test_Combined/202105_5cm_tile_287185_584065.png
/content/drive/MyDrive/train_and _test_data/Week_4_5_dataset/test_Combined/2016_10cm_tile_287185_584065.png
/content/drive/MyDrive/train_and _test_data/Week_4_5_dataset/test_Combined/202105_5cm_tile_286995_583630.png
/conte